# Word2Vec for Tamil words. 

Converting words to vector representation and similarity map generation


In [2]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
import regex
from tqdm import tqdm

In [3]:
import sys

In [4]:
sys.setrecursionlimit(2500)

In [5]:
In_Colab = False
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    In_Colab = True

In [6]:
if In_Colab:
    from google.colab import drive
    drive.mount('/gdrive')

    %cd /gdrive/My\ Drive/Tamizh/data/wiki_data

In [7]:
from constants import *

In [8]:
data = pd.read_csv("../data/tawiki_data_df.csv")

In [9]:
# Shape of the data
data.shape

(234621, 1)

In [10]:
data = data.dropna()

In [11]:
data.shape

(200655, 1)

#### Convert each sentence to list of tokens. 

*observation : this have more then 2 crore vocabulary 
* Need to find the root words  - 

In [12]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


### Drop NA values

### Convert as one long sentence. This will be useful for cleaning vallina migu sorrkal.

In [13]:
one_long_sent = data.Content.str.cat(sep=' ')

## Data Cleaning 

In [14]:
def split_text(text):
    return regex.findall(r'\X', text, regex.U)

def conv_base(text):
    return [split_map.get(k) for k in split_text(text)]

def _lastmap(text):
    if len(text[-1])>1:
        return text[-1][-1]
    return text[-1][0]

def _firstmap(text):
    return text[0][0]

def is_vallmigu_pair(text1,text2):
    try:
        text1 = conv_base(text1)
        text2 = conv_base(text2)
        if (_lastmap(text1) in vall_migu) and (_firstmap(text2) in vall_migu):
            return True
        else:
            return False
    except:
        return False

Remove NaN cols. .

Get list of Unique words from the corpus

In [15]:
one_long_sent = re.sub("\s+", " ", one_long_sent)

In [16]:
one_long_list = one_long_sent.split(' ')

In [17]:
print(f"Length of vocabulary {len(one_long_list)}")

Length of vocabulary 24058566


In [18]:
len(one_long_list)

24058566

In [19]:
one_long_list_len  = len(one_long_list)

In [20]:
one_long_list[1]

'விக்கிப்பீடியா'

In [21]:
vallmigu_pair_set = set()

In [22]:
split_text(one_long_list[2])

['மொ', 'ழி', 'க', 'ள்']

In [23]:

for i in tqdm(range(1,one_long_list_len-2)):
    if is_vallmigu_pair(one_long_list[i],one_long_list[i+1]):
        vallmigu_pair_set.add((one_long_list[i],one_long_list[i+1]))
        one_long_list[i] = ''.join(split_text(one_long_list[i])[:-1])
    

100%|██████████| 24058563/24058563 [10:56<00:00, 36632.15it/s]


In [24]:
len(vallmigu_pair_set)

744945

In [25]:
len(set(one_long_list))

2031806

#Word2Vec

Generate word2vec representation for tokens in tamil. 



In [31]:
import pickle

with open('../data/vallmigu_pair.pkl','wb') as f:
    pickle.dump(vallmigu_pair_set,f)

In [26]:
#Generate word 2 vec

from gensim.models import Word2Vec

In [27]:
texts = one_long_list

In [28]:
 
model = Word2Vec(texts, size=100, window=50, min_count=20, workers=6)

TypeError: __init__() got an unexpected keyword argument 'size'

In [ ]:
len(model.wv.vocab)

80860

In [ ]:
model.wv.most_similar("ஐம்பது")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('முப்பது', 0.7912629842758179),
 ('நூறு', 0.7821017503738403),
 ('நாற்பது', 0.7711770534515381),
 ('பதினைந்து', 0.7171321511268616),
 ('ஆயிரத்து', 0.7063670754432678),
 ('எண்பது', 0.6890194416046143),
 ('பத்து', 0.6858401298522949),
 ('அறுபது', 0.6848776340484619),
 ('இரண்டரை', 0.6679751873016357),
 ('இலட்சம்', 0.6582279205322266)]

In [ ]:
model.save("tawiki_w2v")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
